# Imports

In [15]:
import os
import sqlite3
import pandas as pd
import tensorflow as tf
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
from scipy.special import softmax
import pandas as pd
from concurrent.futures import ThreadPoolExecutor
from sqlalchemy import create_engine
# v1
from tqdm import tqdm
from collections import defaultdict


# Constants

In [2]:
def check_given_var(env_var_str: str) -> str:
    """
    Check if the given environment variable is set and return its value.

    Args:
        env_var_str (str): The name of the environment variable to check.

    Returns:
        str: The value of the environment variable.

    Raises:
        AssertionError: If the environment variable is not found.
    """

    env_var = os.getenv(env_var_str)
    assert (
        env_var is not None
    ), f"{env_var_str} is required but not found in environment variables"
    return env_var


def check_env_vars() -> (str, str, str, str):  # type: ignore
    user = check_given_var("DBL_USER")
    database = check_given_var("DBL_DATABASE")
    password = check_given_var("DBL_PASSWORD")
    host = check_given_var("DBL_HOST")
    return user, database, password, host


USER, DATABASE, PASSWORD, HOST = check_env_vars()
# USER, DATABASE = "nezox2um_test", "nezox2um_test"
QUERY_ALL = """
SELECT 
    Users.user_id AS user_id, 
    Users.creation_time AS user_creation_time, 
    Tweets.creation_time AS tweet_creation_time,
    Tweets.tweet_id,
    Tweets.full_text,
    Tweets.lang,
    Tweets.replied_tweet_id
FROM Users
INNER JOIN Tweets ON Users.user_id = Tweets.user_id;
"""


DTYPES = {
"user_id": "object",
"tweet_id": "object",
"full_text": "object",
"lang": "category",
"replied_tweet_id": "object",
}

COMPANY_NAME_TO_ID = {
    "Klm": "56377143",
    "Air France": "106062176",
    "British Airways": "18332190",
    "American Air": "22536055",
    "Lufthansa": "124476322",
    "Air Berlin": "26223583",
    "Air Berlin assist": "2182373406",
    "easyJet": "38676903",
    "Ryanair": "1542862735",
    "Singapore Airlines": "253340062",
    "Qantas": "218730857",
    "Etihad Airways": "45621423",
    "Virgin Atlantic": "20626359",
}

COMPANY_ID_TO_NAME = {v: k for k, v in COMPANY_NAME_TO_ID.items()}

# Helper functions

In [3]:
def get_local_data(query: str, path: str, dtype: bool = True) -> pd.DataFrame:
    # Connect to the SQLite database using a context manager
    with sqlite3.connect(path) as connection:
        # Read the data into a DataFrame
        if dtype:
            df = pd.read_sql_query(query, connection,
                                   dtype=DTYPES,
                                   index_col='tweet_id')
            df['tweet_creation_time'] = pd.to_datetime(df['tweet_creation_time'])
            df['user_creation_time'] = pd.to_datetime(df['user_creation_time'])
        else:
            df = pd.read_sql_query(query, connection)
    
    return df


def fetch_data(query: str, dtype: bool = True) -> pd.DataFrame:
    engine = create_engine(f"mysql://{USER}:{PASSWORD}@{HOST}:3306/{DATABASE}")
    if dtype:
        return pd.read_sql_query(query, engine,
                                 dtype=DTYPES, index_col='tweet_id')
    return pd.read_sql_query(query, engine)


# Loading

In [4]:
# Server
# test_data = fetch_data(QUERY_ALL)
# Local
path =  os.path.join(
    os.path.dirname(
        os.path.dirname(
            os.getcwd()
            )
        ),
    "data_processed", "local_backup.db")

test_data = get_local_data(QUERY_ALL, path)

# Conversations

In [5]:
convo_special = test_data[["user_id", "replied_tweet_id", "tweet_creation_time"]]
convo_special

,user_id,replied_tweet_id,tweet_creation_time
tweet_id,,,
1131172858951024641,393374091,None,2019-05-22 12:20:00+00:00
1130922003702177800,880417607865815040,1130615560910254080,2019-05-21 19:43:11+00:00
1131172864147808257,3420691215,None,2019-05-22 12:20:01+00:00
1131172867985485824,394376606,1131032916232826881,2019-05-22 12:20:02+00:00
1131030279278063616,227687574,None,2019-05-22 02:53:26+00:00
...,...,...,...
1244696703690772485,278698748,None,2020-03-30 18:43:14+00:00
1244696708983984131,246520593,None,2020-03-30 18:43:15+00:00
1244696710447800320,109284383,None,2020-03-30 18:43:15+00:00


In [6]:
class TrieNode:
    def __init__(self):
        self.children = defaultdict(TrieNode)
        self.is_end = False

class Trie:
    def __init__(self):
        self.root = TrieNode()

    def insert(self, conversation):
        node = self.root
        for tweet_id in conversation:
            node = node.children[tweet_id]
        node.is_end = True

    def is_subset(self, conversation):
        node = self.root
        for tweet_id in conversation:
            if tweet_id not in node.children:
                return False
            node = node.children[tweet_id]
        return True

def trace_conversation(start_tweet_id: str, tweet_dict: dict):
    convo = []
    current_tweet_id = start_tweet_id
    users_in_conversation = set()
    local_processed_tweet_ids = set()  # Local set to track the current conversation
    while current_tweet_id:
        if current_tweet_id not in tweet_dict or current_tweet_id in local_processed_tweet_ids:
            break
        tweet_info = tweet_dict[current_tweet_id]
        convo.append(current_tweet_id)
        users_in_conversation.add(tweet_info['user_id'])
        local_processed_tweet_ids.add(current_tweet_id)
        if len(users_in_conversation) > 2:
            return convo[:-1][::-1]  # As soon as the third user appears, we delete his tweet and return
        current_tweet_id = tweet_info['replied_tweet_id']
    return convo[::-1] if len(users_in_conversation) == 2 else None

def extract_and_filter_conversations(df: pd.DataFrame):
    df = df.sort_values("tweet_creation_time", ascending=False)
    df.index = df.index.astype(str)
    tweet_dict = df.to_dict('index')
    conversations = []
    trie = Trie()  # Initialize trie for subset checks

    # Start tracing conversations from tweets that are replies
    for tweet_id in tqdm(df[df['replied_tweet_id'].notnull()].index, desc="Extracting all conversations"):
        if conversation := trace_conversation(tweet_id, tweet_dict):
            if not trie.is_subset(conversation):
                trie.insert(conversation)
                conversations.append(conversation)

    return conversations



In [7]:
conversations = extract_and_filter_conversations(convo_special)

Extracting all conversations: 100%|██████████| 1795409/1795409 [00:07<00:00, 256295.28it/s]


In [8]:
data = []
for convo_num, convo in enumerate(conversations, start=1):
    data.extend((convo_num, tweet_id) for tweet_id in convo)
# Create a DataFrame
df_conversations = pd.DataFrame(data, columns=['Conversation', 'Tweet_ID'])

# Set MultiIndex
df_conversations

,Conversation,Tweet_ID
0,1,1244694453190897664
1,1,1244696682979303426
2,2,1244677304598609923
3,2,1244696641401163776
4,3,1244648694454026240
...,...,...
2712242,1064150,451125255294443521
2712243,1064151,430790355962052608
2712244,1064151,430792524043931648
2712245,1064152,248528541157834752


In [9]:

# Merge the conversation DataFrame with the test_data DataFrame
df_conversations_full = df_conversations.merge(test_data, left_on='Tweet_ID', right_index=True, how='left')

# Set the MultiIndex again with Conversation and Tweet_ID
df_conversations_full.set_index(['Conversation', 'Tweet_ID'], inplace=True)
df_conversations_full


user_id  \
Conversation Tweet_ID                                   
1            1244694453190897664            521835883   
             1244696682979303426             20626359   
2            1244677304598609923            396021583   
             1244696641401163776   832964639436701696   
3            1244648694454026240  1233410199500791809   
...                                               ...   
1064150      451125255294443521              22536055   
1064151      430790355962052608              64327804   
             430792524043931648              22536055   
1064152      248528541157834752              19911051   
             248529937198366720             399494759   

                                        user_creation_time  \
Conversation Tweet_ID                                        
1            1244694453190897664 2012-03-12 01:11:22+00:00   
             1244696682979303426 2009-02-11 20:50:56+00:00   
2            1244677304598609923 2011-10-22 16:35:05+00:00   
             1244696641401163776 2017-02-18 14:47:00+00:00   
3            1244648694454026240 2020-02-28 15:14:56+00:00   
...                                                    ...   
1064150      451125255294443521  2009-03-02 21:23:05+00:00   
1064151      430790355962052608  2009-08-10 03:34:27+00:00   
             430792524043931648  2009-03-02 21:23:05+00:00   
1064152      248528541157834752  2009-02-02 15:17:02+00:00   
             248529937198366720  2011-10-27 15:47:03+00:00   

                                       tweet_creation_time  \
Conversation Tweet_ID                                        
1            1244694453190897664 2020-03-30 18:34:17+00:00   
             1244696682979303426 2020-03-30 18:43:09+00:00   
2            1244677304598609923 2020-03-30 17:26:09+00:00   
             1244696641401163776 2020-03-30 18:42:59+00:00   
3            1244648694454026240 2020-03-30 15:32:27+00:00   
...                                                    ...   
1064150      451125255294443521  2014-04-01 22:33:37+00:00   
1064151      430790355962052608  2014-02-04 19:49:59+00:00   
             430792524043931648  2014-02-04 19:58:36+00:00   
1064152      248528541157834752  2012-09-19 21:06:36+00:00   
             248529937198366720  2012-09-19 21:12:09+00:00   

                                                                          full_text  \
Conversation Tweet_ID                                                                 
1            1244694453190897664  @nealrach @VirginAtlantic Siiiigh.... Still no...   
             1244696682979303426  @Jade_Velveteese Hi Jade. We have an ‘Away fro...   
2            1244677304598609923  @VirginAtlantic Sod off your primary sharehold...   
             1244696641401163776  @Boyde11 @VirginAtlantic Get your facts right,...   
3            1244648694454026240  @flavioArCab @Chapux0204 @chechiffss @aeronaut...   
...                                                                             ...   
1064150      451125255294443521   @lanaupdates_ Your information has been forwar...   
1064151      430790355962052608   @AmericanAir phew, they finally turned on the ...   
             430792524043931648   @benjy_greenberg It looks like we'll have you ...   
1064152      248528541157834752   Un-fucking believable!\nThanks @BritishAirways...   
             248529937198366720   @djmarkknight @britishairways i have have the ...   

                                 lang     replied_tweet_id  
Conversation Tweet_ID                                       
1            1244694453190897664   en  1243885949697888263  
             1244696682979303426   en  1244694453190897664  
2            1244677304598609923   en  1244669964289806338  
             1244696641401163776   en  1244677304598609923  
3            1244648694454026240   es  1244643427515535360  
...                               ...                  ...  
1064150      451125255294443521    en   451124070730719233  
1064151  

In [10]:
airline_conversation = df_conversations_full.loc[df_conversations_full.index.get_level_values('Conversation').isin(df_conversations_full[df_conversations_full['user_id'] == COMPANY_NAME_TO_ID["Lufthansa"]].index.get_level_values('Conversation'))]
airline_conversation

user_id        user_creation_time  \
Conversation Tweet_ID                                                    
140          1244658051740774400   124476322 2010-03-19 14:30:32+00:00   
             1244688840033546245   119901222 2010-03-04 22:18:43+00:00   
             1244691330837762051   124476322 2010-03-19 14:30:32+00:00   
194          1244684192040071173    62555545 2009-08-03 16:35:21+00:00   
             1244688444162486273   124476322 2010-03-19 14:30:32+00:00   
...                                      ...                       ...   
1064013      1095473044573700096  1339792290 2013-04-09 17:56:30+00:00   
1064019      1090912573304774662   124476322 2010-03-19 14:30:32+00:00   
             1090913775417479168    18631142 2009-01-05 13:13:10+00:00   
1064028      1083376699780280321   478699784 2012-01-30 15:32:40+00:00   
             1083381532071469058   124476322 2010-03-19 14:30:32+00:00   

                                       tweet_creation_time  \
Conversation Tweet_ID                                        
140          1244658051740774400 2020-03-30 16:09:38+00:00   
             1244688840033546245 2020-03-30 18:11:59+00:00   
             1244691330837762051 2020-03-30 18:21:53+00:00   
194          1244684192040071173 2020-03-30 17:53:31+00:00   
             1244688444162486273 2020-03-30 18:10:24+00:00   
...                                                    ...   
1064013      1095473044573700096 2019-02-13 00:01:41+00:00   
1064019      1090912573304774662 2019-01-31 10:00:00+00:00   
             1090913775417479168 2019-01-31 10:04:46+00:00   
1064028      1083376699780280321 2019-01-10 14:55:08+00:00   
             1083381532071469058 2019-01-10 15:14:20+00:00   

                                                                          full_text  \
Conversation Tweet_ID                                                                 
140          1244658051740774400  @thick_daddy The online cancellation tool will...   
             1244688840033546245  @lufthansa @thick_daddy I received the same em...   
             1244691330837762051  @NateThomasNOLA @thick_daddy At the moment, my...   
194          1244684192040071173  @lufthansa had an email stating changes to my ...   
             1244688444162486273  @Holgate1987 At the moment, my colleagues in t...   
...                                                                             ...   
1064013      1095473044573700096  @lufthansa Read on @YahooNews you sued a guy f...   
1064019      1090912573304774662  With 297 seats, the #A340-600 is next in line ...   
             1090913775417479168      @lufthansa 747-400, then 747-8, then A380. :)   
1064028      1083376699780280321  Hi @lufthansa how come there is no 'Ms' option...   
             1083381532071469058  @JosephPKilroy The "Ms" option is supposed to ...   

                                 lang     replied_tweet_id  
Conversation Tweet_ID                                       
140          1244658051740774400   en  1244650317494566913  
             1244688840033546245   en  1244658051740774400  
             1244691330837762051   en  1244688840033546245  
194          1244684192040071173   en                 None  
             1244688444162486273   en  1244684192040071173  
...                               ...                  ...  
1064013      1095473044573700096   en  1094985662384594944  
1064019      1090912573304774662   en                 None  
             1090913775417479168   en  1090912573304774662  
1064028      1083376699780280321   en                 None  
             1083381532071469058   en  1083376699780280321  

[44091 rows x 6 columns]

In [11]:
airline_conversation = airline_conversation.reset_index()
airline_conversation['New_Conversation'] = pd.factorize(airline_conversation['Conversation'])[0] + 1
airline_conversation = airline_conversation.set_index(['New_Conversation', 'Tweet_ID'])
airline_conversation = airline_conversation.sort_index(level='New_Conversation')
airline_conversation

Conversation     user_id  \
New_Conversation Tweet_ID                                        
1                1244658051740774400           140   124476322   
                 1244688840033546245           140   119901222   
                 1244691330837762051           140   124476322   
2                1244684192040071173           194    62555545   
                 1244688444162486273           194   124476322   
...                                            ...         ...   
14271            1095473044573700096       1064013  1339792290   
14272            1090912573304774662       1064019   124476322   
                 1090913775417479168       1064019    18631142   
14273            1083376699780280321       1064028   478699784   
                 1083381532071469058       1064028   124476322   

                                            user_creation_time  \
New_Conversation Tweet_ID                                        
1                1244658051740774400 2010-03-19 14:30:32+00:00   
                 1244688840033546245 2010-03-04 22:18:43+00:00   
                 1244691330837762051 2010-03-19 14:30:32+00:00   
2                1244684192040071173 2009-08-03 16:35:21+00:00   
                 1244688444162486273 2010-03-19 14:30:32+00:00   
...                                                        ...   
14271            1095473044573700096 2013-04-09 17:56:30+00:00   
14272            1090912573304774662 2010-03-19 14:30:32+00:00   
                 1090913775417479168 2009-01-05 13:13:10+00:00   
14273            1083376699780280321 2012-01-30 15:32:40+00:00   
                 1083381532071469058 2010-03-19 14:30:32+00:00   

                                           tweet_creation_time  \
New_Conversation Tweet_ID                                        
1                1244658051740774400 2020-03-30 16:09:38+00:00   
                 1244688840033546245 2020-03-30 18:11:59+00:00   
                 1244691330837762051 2020-03-30 18:21:53+00:00   
2                1244684192040071173 2020-03-30 17:53:31+00:00   
                 1244688444162486273 2020-03-30 18:10:24+00:00   
...                                                        ...   
14271            1095473044573700096 2019-02-13 00:01:41+00:00   
14272            1090912573304774662 2019-01-31 10:00:00+00:00   
                 1090913775417479168 2019-01-31 10:04:46+00:00   
14273            1083376699780280321 2019-01-10 14:55:08+00:00   
                 1083381532071469058 2019-01-10 15:14:20+00:00   

                                                                              full_text  \
New_Conversation Tweet_ID                                                                 
1                1244658051740774400  @thick_daddy The online cancellation tool will...   
                 1244688840033546245  @lufthansa @thick_daddy I received the same em...   
                 1244691330837762051  @NateThomasNOLA @thick_daddy At the moment, my...   
2                1244684192040071173  @lufthansa had an email stating changes to my ...   
                 1244688444162486273  @Holgate1987 At the moment, my colleagues in t...   
...                                                                                 ...   
14271            1095473044573700096  @lufthansa Read on @YahooNews you sued a guy f...   
14272            1090912573304774662  With 297 seats, the #A340-600 is next in line ...   
                 1090913775417479168      @lufthansa 747-400, then 747-8, then A380. :)   
14273            1083376699780280321  Hi @lufthansa how come there is no 'Ms' option...   
                 1083381532071469058  @JosephPKilroy The "Ms" option is supposed to ...   

                                     lang     replied_tweet_id  
New_Conversation Tweet_ID                                       
1                1244658051740774400   en  1244650317494566913  
                 1244688840033546245   en  1244658051740774400  
                 

# Sentiment Analysis

In [12]:
#Load the tokenizer and model once
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment")

#Check if GPU is available
if tf.test.is_gpu_available():
    device = '/GPU:0'
else:
    device = '/CPU:0'

#Set the labels for sentiment results
labels = {
    0 : 'negative',
    1 : 'neutral',
    2 : 'positive'
}

def process_batch(texts):
    """
    Apply sentiment analysis to a batch of texts using a pre-trained transformer model.

    Parameters:
    texts (list): A list of texts to analyze.

    Returns:
    list: A list of ranked sentiment labels for the input texts.

    This function uses a pre-trained transformer model for sequence classification to analyze the sentiment of the texts in the given batch. It returns a list of ranked sentiment labels for the input texts, where the first label is the most likely sentiment and the subsequent labels are less likely sentiments.
    """
    encoded_input = tokenizer(texts, return_tensors='tf', padding=True, truncation=True)
    with tf.device(device):
        output = model(encoded_input)

    scores = output[0].numpy()
    scores = softmax(scores, axis=1)

    sentiment_scores = scores[:, 2] - scores[:, 0]
    return sentiment_scores.tolist()

def apply_sentiment_analysis(df, text_column, batch_size=64, max_workers=4):
    """
    Apply sentiment analysis to the given DataFrame using a pre-trained transformer model.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the text column to analyze.
    text_column (str): The name of the column in the DataFrame containing the text to analyze.
    batch_size (int): The number of texts to process in each batch. Default is 128.
    max_workers (int): The maximum number of worker threads to use for parallel processing. Default is 4.

    Returns:
    pd.DataFrame: The input DataFrame with an additional 'sentiment' column containing the sentiment analysis results.

    This function uses a pre-trained transformer model for sequence classification to analyze the sentiment of the texts in the given DataFrame. It applies the sentiment analysis in parallel using multiple worker threads to improve performance. The results are then added to the input DataFrame as a new 'sentiment' column.
    """
    texts = df[text_column].tolist()
    results = []

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        for i in range(0, len(texts), batch_size):
            batch = texts[i:i+batch_size]
            results.extend(executor.submit(process_batch, batch).result())

    df['sentiment'] = results
    return df


c:\My programs\Python 3.12\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


ImportError: 
TFAutoModelForSequenceClassification requires the TensorFlow library but it was not found in your environment. Checkout the instructions on the
installation page: https://www.tensorflow.org/install and follow the ones that match your environment.
Please note that you may need to restart your runtime after installation.


In [ ]:
pd.set_option('display.max_colwidth', None)  
full_text_df = airline_conversation[['full_text']].copy()
airline_conversation_sample = full_text_df.head(5)
airline_conversation_sample = apply_sentiment_analysis(airline_conversation_sample, 'full_text')
airline_conversation_sample


In [ ]:
import mysql.connector
from mysql.connector import Error
from typing import Tuple, List
import pandas as pd
from tqdm import tqdm


def connect_to_database(user: str, database: str, password: str, host: str):
    """
    Establish a connection to the database.
    
    Args:
        user: The database user.
        database: The name of the database.
        password: The password for the database.
        host: The database host.
    
    Returns:
        A connection object to the MySQL database.
    """
    try:
        connection = mysql.connector.connect(
            user=user,
            password=password,
            host=host,
            database=database
        )
        if connection.is_connected():
            return connection
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    return None


def get_batches(df: pd.DataFrame, batch_size: int = 1000) -> List[pd.DataFrame]:
    """
    Split DataFrame into batches of DataFrames with specified batch size.
    
    Args:
        df: The DataFrame containing tweet data.
        batch_size: The size of each batch.
        
    Returns:
        A list of DataFrames, each containing a batch of rows.
    """
    return [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]

def update_sentiment_batch(user: str, database: str, password: str, host: str, batch: List[Tuple[str, float]]):
    """
    Update sentiment values for a batch of data in the database.
    
    Args:
        user: The database user.
        database: The name of the database.
        password: The password for the database.
        host: The database host.
        batch: A list of tuples containing tweet_id and sentiment.
    """
    try:
        connection = connect_to_database(user, database, password, host)
        if connection is None:
            return
        cursor = connection.cursor()
        update_query = "UPDATE Tweets SET sentiment_score = %s WHERE tweet_id = %s"
        cursor.executemany(update_query, batch)
        connection.commit()
        cursor.close()
        connection.close()
    except Error as e:
        print(f"Error updating batch: {e}")

def upload_sentiment(batch: Tuple[Tuple[float, str]],
                     user: str, database: str, password: str,
                     host: str) -> None:
    """
    Create and insert batches of tweets into the database in parallel.
    
    Args:
        batches_list: Tuple of (sentiment, tweet_id) pairs.
        user: The database user.
        database: The name of the database.
        password: The password for the database.
        host: The database host.
    """
    connection = connect_to_database(user, database, password, host)
    if connection is None:
        return
    cursor = connection.cursor()
    update_query = "UPDATE Tweets SET sentiment_score = %s WHERE tweet_id = %s"
    cursor.executemany(update_query, batch)
    connection.commit()
    cursor.close()
    connection.close()

    print("Sentiment values updated successfully")

In [ ]:
def convert_to_list(df: pd.DataFrame) -> List[List]:
    """
    Convert DataFrame with tweet_id as index to a list of lists containing sentiment and tweet_id.
    
    Args:
        df: The DataFrame with tweet_id as index and sentiment as a column.
        
    Returns:
        A list of lists containing tweet_id and sentiment.
    """
    tweet_ids = df.index.to_numpy()
    sentiments = df['sentiment'].to_numpy()
    return tuple(zip(sentiments, tweet_ids))





In [ ]:
data_batches = get_batches(test_data[["full_text"]], 10_000)

In [ ]:
user, database, password, host = os.getenv("DBL_USER"), os.getenv("DBL_DATABASE"), os.getenv("DBL_PASSWORD"), os.getenv("DBL_HOST")
for batch in tqdm(data_batches):
    sentiment_with_score = apply_sentiment_analysis(batch, "full_text")
    upload_sentiment(convert_to_list(sentiment_with_score), user, database, password, host)